# Assignment 3 ( Question 4) Know the hatred!

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- all the packages that are imported in the template code

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q4.ipynb`.


In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import sklearn
import torch


In [4]:
!pip install transformers
import transformers
from transformers import AutoTokenizer, AutoModel

#Dataset

In the training data, the comments are labelled as one or more of the six categories; toxic, severe toxic, obscene, threat, insult and identity hate. This is essentially a multi-label classification problem.

The dataset here is a multi-label classification dataset. To understand multi-label classification datasets, you can refer here: https://www.analyticsvidhya.com/blog/2017/08/introduction-to-multi-label-classification/ 

.


Here, we will be using Binary relevance to solve our multi-label classification problem



**Binary Relevance**: This is probably the simplest which treats each label as a separate single classification problems. The key assumption here though, is that there are no correlation among the various labels.

Finally, we will be summing up the accuracies obtained of all labels to obtain the final accuracy


In [9]:
#REMOVE LATER
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My Drive/smai/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/smai


In [10]:
import os
os.getcwd()

'/content/drive/MyDrive/smai'

In [11]:
# loading the dataset
train_df = pd.read_csv('q4_data/train.csv')
test_df = pd.read_csv('q4_data/test.csv')
test_labels = pd.read_csv('q4_data/test_labels.csv')
test_labels = test_labels[test_labels.toxic != -1]
test_df = pd.merge(test_df, test_labels, how="inner", on='id')
#dataset understanding
train_df.sample(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
111693,557d3b98387207ae,Gotcha. Will take a look.,0,0,0,0,0,0
62370,a6e95e0d451296f7,"I'm playing around with using a timeline box, ...",0,0,0,0,0,0
93724,faa41553e558620d,"I'm sorry, that was rude. Thanks for the welco...",0,0,0,0,0,0
43048,72d8591b73fa55a0,I have accepted your apology. Maybe our paths ...,0,0,0,0,0,0
38294,663507cf33a91f52,JAN MOIR IS A VILE SLAG- SACK THE BITCH FROM T...,1,0,0,0,0,0


In [12]:
# let's see the total rows in train, test data and the numbers for the various categories
cols_target=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
print('Total rows in test is {}'.format(len(test_df)))
print('Total rows in train is {}'.format(len(train_df)))
print(train_df[cols_target].sum())


Total rows in test is 63978
Total rows in train is 159571
toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64


In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text


In [14]:
train_df['comment_text'] = train_df['comment_text'].map(lambda com : clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com : clean_text(com))

In [15]:
# train_df = train_df.drop('char_length',axis=1)
X = train_df.comment_text
test_X = test_df.comment_text
print(X.shape, test_X.shape)


(159571,) (63978,)


Now, we will be experimenting this classification problem with different classifiers. We will be using pre-trained models inorder to obtain the required embeddings. Here, we will be using BERT and Universal Sentence Encoder embeddings.


Understand how Elmo and USE(Universal Sentence Encoder) works here. You can use any of both to train your classifiers.

To understand how different sentence embeddings work, you can refer to this link: https://neptune.ai/blog/how-to-code-bert-using-pytorch-tutorial 

Please give it a look before you start working with transfer learning/ sentence embeddings like BERT, Elmo, Ulm-fit, USE.


In [16]:
#loading universal sentence encoder

import tensorflow_hub as hub
# Load pre-trained universal sentence encoder model
use_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# elmo_embed = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [17]:
def get_use_embedding(sentence):
  embedding = use_embed([sentence])
  embedding = embedding.numpy()
  return embedding[0]

In [18]:
#to obtain bert embeddings ( you need not completely understand how it works)
#you can directly use the function get_bert_embedding to obtain the embedding


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = model.to(device)


def get_bert_embedding(sentence):
  encoded_input = tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
  with torch.no_grad():
    model_output = model(**encoded_input)
  sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embedding[0]


Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
#Obtain X_test and X_train 

In [ ]:
X_train = []

for i in range(len(X)):
  val = get_bert_embedding(X[i]).cpu().numpy()
  X_train.append(val)

X_train = np.array(X_train)

In [ ]:
X_test = []

for i in range(len(test_X)):
  val = get_bert_embedding(test_X[i]).cpu().numpy()
  X_test.append(val)

X_test = np.array(X_test)

In [23]:
np.save('X_train',X_train)
np.save('X_test',X_test)

In [24]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')

In [25]:
#change
X_train=X_train
X_test=X_test
print(X_train.shape)
print(X_test.shape)

(159571, 768)
(63978, 768)


In [26]:
Y_train = (train_df[cols_target])
Y_test = (test_df[cols_target])

In [27]:
Y1=np.array(Y_train['toxic']).reshape(Y_train.shape[0],1)
Y2=np.array(Y_train['severe_toxic']).reshape(Y_train.shape[0],1)
Y3=np.array(Y_train['obscene']).reshape(Y_train.shape[0],1)
Y4=np.array(Y_train['threat']).reshape(Y_train.shape[0],1)
Y5=np.array(Y_train['insult']).reshape(Y_train.shape[0],1)
Y6=np.array(Y_train['identity_hate']).reshape(Y_train.shape[0],1)
print(Y1.shape)
print(Y2.shape)

(159571, 1)
(159571, 1)


In [ ]:
X_train_copy=X_train.T
X_test_copy=X_test.T
Y_train_copy=Y_train.T
Y_test_copy=Y_test.T

In [ ]:
print(X_train_copy.shape)
print(X_test_copy.shape)
print(Y_train_copy.shape)
print(Y_test_copy.shape)

(768, 159571)
(768, 63978)
(6, 159571)
(6, 63978)


# Classifiers
Here, we use different classifiers for our classification problem. You can directly import classifiers for first two.


*   Naive Bayes classifier
*   Support Vector Machine
*   Multi-layer Perceptron(MLP)

In [28]:
!pip install scikit-multilearn
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

     |████████████████████████████████| 89 kB 3.8 MB/s 


In [ ]:
#calculate the accuracies obtained, u can use sklearn metrics


In [ ]:
# Naive Bayes classifier 
naive_bayes = BinaryRelevance(GaussianNB())
y_pred = naive_bayes.fit(np.array(X_train), np.array(Y_train)).predict(np.array(X_test))
accuracy_score(Y_test, y_pred)

0.7384413392103536

In [ ]:
SVM = BinaryRelevance(classifier=LinearSVC(),require_dense=[False,True])

# train
SVM.fit(X_train,np.array(Y_train))

# predict
predictions = SVM.predict(X_test)
accuracy_score(Y_test, predictions)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

0.8795054550001563

Which classifier of the three performed better, and why do you think so ?

Ans: In almost every situation, MLP will outperform. Naive Bayes regards features as separate entities, whereas SVM considers their interactions to some extent. Pre-defined kernels are commonly used in SVMs, and they usually work well. A SVM, on the other hand, will perform worse than an MLP on situations where it is best to let an MLP generate its own non-linear projection due to its one-size-fits-all nature.


#Multi-layer Perceptron

For Multi-layer Perceptron, first you will be implementing your own articial Neural network, and then compare it with sklearn's MLP layer.


Implement your own artificial Neural Network (MLP)
Few Steps to be followed(for your reference):

*   Initialise the weights randomly
*   Decide upon the number of layers you wanna have and number of neurons associated in each layer. ( Have a clear idea of what will be the size of the weight matrices).
*   Write the code for forward and backward propogations
*   Decide upon the loss function
*   Train the model
*   Predict the labels after training the model




In [ ]:
# you can remove returns statements if u think they are not required
# you can also add functions if needed
def sigmoid(z):
  return 1./(1.+np.exp(-z))

class SimpleNeuralNet:
  #ARCH:
  # layer 1(inp-dim -> hidden_layer) -> tanH -> hidden_layer(hidden_layer -> 1)->sigmoid 
  def __init__(self,inp_size,hidden_size,out_size, learning_rate=1.2,epochs=10):
    #Initialize random weights with seed
    np.random.seed(2)
    self.weight_1 = np.random.randn(hidden_size,inp_size) * 0.01
    self.bias_1 = np.zeros((hidden_size,1))
    self.weight_2= np.random.randn(out_size,hidden_size) * 0.01
    self.bias_2 = np.zeros((out_size,1))
    self.learning_rate=learning_rate
    self.epochs=epochs
  def summary(self):
    print(f'First layer dimensions:{self.weight_1.shape}')
    print(f'Hidden layer dimensions:{self.weight_2.shape}')
    print(f'learning rate:{self.learning_rate}')
    print(f'Epochs:{self.epochs}')
  def loss_function(self,activation_out, Y):
    #cross entropy loss
    m = Y.shape[1]
    logprobs = np.multiply(np.log(activation_out),Y) + np.multiply( (1-Y), np.log(1-activation_out) )
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
    return cost
  def forward_layer(self,X):
    self.out_1 = np.dot(self.weight_1,X) + self.bias_1
    self.activation_out_1 = np.tanh(self.out_1)
    self.out_2 = np.dot(self.weight_2,self.activation_out_1) + self.bias_2
    self.activation_out_2 = sigmoid(self.out_2)
    return self.activation_out_2
  def backward_propogation(self,X,Y):
    m = X.shape[1]
    #calculating the gradients
    dZ_2=self.activation_out_2 - Y
    dweight_2 = (1/m) * np.dot(dZ_2,self.activation_out_1.T)
    #hidden layer gradients
    dbias_2 = (1/m) * np.sum(dZ_2, axis=1, keepdims=True)
    dZ_1 = np.dot(self.weight_2.T,dZ_2) * (1 - np.power(self.activation_out_1,2))
    #first layer gradients
    dweight_1 = (1/m) * np.dot(dZ_1,X.T)
    dbias_1 = (1/m) * np.sum(dZ_1, axis=1, keepdims=True)
    #update gradients:
    self.weight_1=self.weight_1 - self.learning_rate * dweight_1
    self.weight_2=self.weight_2 - self.learning_rate *dweight_2
    self.bias_1=self.bias_1 - self.learning_rate*dbias_1
    self.bias_2=self.bias_2 - self.learning_rate*dbias_2
  def train(self,X,Y):
    losses=[]
    for i in range(0,self.epochs):
      out=self.forward_layer(X)
      loss=self.loss_function(out,Y)
      self.backward_propogation(X,Y)
      losses.append(loss)
    return losses
  def predict(self,X):
    out=self.forward_layer(X)
    pred=np.round(out)
    return pred

In [ ]:
input_dim=X_train_copy.shape[0]
output_dim=Y1.T.shape[0]
print(input_dim)
print(output_dim)

768
1


In [ ]:
nn=SimpleNeuralNet(input_dim,4,output_dim,epochs=20)
nn.summary()

First layer dimensions:(4, 768)
Hidden layer dimensions:(1, 4)
learning rate:1.2
Epochs:20


In [ ]:
nn.train(X_train_copy,Y1.T)

[0.692986728215387,
 0.5179423697848703,
 0.40110861571286704,
 0.3338338903019544,
 0.31618153280171457,
 0.31107407687555383,
 0.3012109705051856,
 0.2664335260685963,
 0.27609306887720836,
 0.3272251344609271,
 0.2901600708154749,
 0.2523040856814907,
 0.22873285594995518,
 0.21751433134361642,
 0.21832693408383788,
 0.26862188046205016,
 0.34687108612313966,
 0.2983906149264826,
 0.24940460804438278,
 0.21167636130136022]

In [ ]:
output=nn.predict(X_test_copy)
print(output)

[[0. 1. 0. ... 0. 1. 0.]]


In [ ]:
nn1=SimpleNeuralNet(input_dim,4,output_dim)
nn2=SimpleNeuralNet(input_dim,4,output_dim)
nn3=SimpleNeuralNet(input_dim,4,output_dim)
nn4=SimpleNeuralNet(input_dim,4,output_dim)
nn5=SimpleNeuralNet(input_dim,4,output_dim)
nn6=SimpleNeuralNet(input_dim,4,output_dim)

In [ ]:
nn1.train(X_train_copy,Y1.T)
nn2.train(X_train_copy,Y2.T)
nn3.train(X_train_copy,Y3.T)
nn4.train(X_train_copy,Y4.T)
nn5.train(X_train_copy,Y5.T)
nn6.train(X_train_copy,Y6.T)

[0.6929414671062459,
 0.434302597043537,
 0.2603213394785661,
 0.13308431489131728,
 0.09308448013228783,
 0.07793588435486105,
 0.06947802905379363,
 0.06425669291531755,
 0.060824328980183875,
 0.058451234955060764]

In [ ]:
pred1=nn1.predict(X_test_copy).T
pred2=nn2.predict(X_test_copy).T
pred3=nn3.predict(X_test_copy).T
pred4=nn4.predict(X_test_copy).T
pred5=nn5.predict(X_test_copy).T
pred6=nn6.predict(X_test_copy).T

In [ ]:
#report accuracies on the test dataset 
from sklearn.metrics import accuracy_score
output=np.hstack((pred1,pred2,pred3,pred4,pred5,pred6))
print(output.shape)
score=accuracy_score(Y_test, output)
print(f'Score:{score}')

(63978, 6)
Score:0.90241958173122


In [ ]:
print(X_train.shape)
print(Y_train.shape)

(159571, 768)
(159571, 6)


Now, compare your implemented MLP with the sklearn MLP layer.

In [ ]:
#report accuracies both on train dataset and test dataset

from sklearn.neural_network import MLPClassifier
MLP=MLPClassifier(random_state=1, max_iter=1000).fit(X_train, Y_train)
MLP.score(X_test, Y_test)

0.8574041076620088

Q. Which MLP performed better? Understand and Analyse.

Ans. MLP from scratch performed better as the threshold in predict api was manually configured to be 0.5, whereas MLP Layer from Sk learn does not have this implementation.